In [ ]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
from torch.nn import Parameter
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
import numpy as np
import scipy.sparse as sp

def get_spectral_rad(sparse_tensor, tol=1e-5):
    """Compute spectral radius from a tensor"""
    A = sparse_tensor.data.coalesce().cpu()
    A_scipy = sp.coo_matrix((np.abs(A.values().numpy()), A.indices().numpy()), shape=A.shape)
    return np.abs(sp.linalg.eigs(A_scipy, k=1, return_eigenvectors=False)[0]) + tol

def projection_norm_inf(A, kappa=0.99, transpose=False):
    """ project onto ||A||_inf <= kappa return updated A"""
    # TODO: speed up if needed
    v = kappa
    if transpose:
        A_np = A.T.clone().detach().cpu().numpy()
    else:
        A_np = A.clone().detach().cpu().numpy()
    x = np.abs(A_np).sum(axis=-1)
    for idx in np.where(x > v)[0]:
        # read the vector
        a_orig = A_np[idx, :]
        a_sign = np.sign(a_orig)
        a_abs = np.abs(a_orig)
        a = np.sort(a_abs)

        s = np.sum(a) - v
        l = float(len(a))
        for i in range(len(a)):
            # proposal: alpha <= a[i]
            if s / l > a[i]:
                s -= a[i]
                l -= 1
            else:
                break
        alpha = s / l
        a = a_sign * np.maximum(a_abs - alpha, 0)
        # verify
        assert np.isclose(np.abs(a).sum(), v, atol=1e-4)
        # write back
        A_np[idx, :] = a
    A.data.copy_(torch.tensor(A_np.T if transpose else A_np, dtype=A.dtype, device=A.device))
    return A


def sparse_mx_to_torch_sparse_tensor(sparse_mx, device=None):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    tensor = torch.sparse.FloatTensor(indices, values, shape)
    if device is not None:
        tensor = tensor.to(device)
    return tensor

def fetch_normalization(type):
   switcher = {
       'AugNormAdj': aug_normalized_adjacency,  # A' = (D + I)^-1/2 * ( A + I ) * (D + I)^-1/2
   }
   func = switcher.get(type, lambda: "Invalid normalization technique.")
   return func

def aug_normalized_adjacency(adj, need_orig=False):
   if not need_orig:
       adj = adj + sp.eye(adj.shape[0])
   adj = sp.coo_matrix(adj)
   row_sum = np.array(adj.sum(1))
   d_inv_sqrt = np.power(row_sum, -0.5).flatten()
   d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
   d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
   return d_mat_inv_sqrt.dot(adj).dot(d_mat_inv_sqrt).tocoo()

def row_normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def preprocess_citation(adj, features, normalization="FirstOrderGCN"):
    adj_normalizer = fetch_normalization(normalization)
    adj = adj_normalizer(adj)
    features = row_normalize(features)
    return adj, features


def load_citation_chain(normalization, cuda, need_orig=False):
    """load the synthetic dataset: chain"""
    r = np.random.RandomState(42)
    c = 2 # num of classes
    n = 20 # chains for each class
    l = 10 # length of chain
    f = 100 # feature dimension
    tn = 20  # train nodes
    vl = 100 # val nodes
    tt = 200 # test nodes
    noise = 0.00

    
    chain_adj = sp.coo_matrix((np.ones(l-1), (np.arange(l-1), np.arange(1, l))), shape=(l, l))
    adj = sp.block_diag([chain_adj for _ in range(c*n)]) # square matrix N = c*n*l

    features = r.uniform(-noise, noise, size=(c, n, l, f))
    #features = np.zeros_like(features)
    features[:, :, 0, :c] += np.eye(c).reshape(c, 1, c) # add class info to the first node of chains.
    features = features.reshape(-1, f)

    labels = np.eye(c).reshape(c, 1, 1, c).repeat(n, axis=1).repeat(l, axis=2) # one-hot labels
    labels = labels.reshape(-1, c)

    idx_random = np.arange(c*n*l)
    r.shuffle(idx_random)
    idx_train = idx_random[:tn]
    idx_val = idx_random[tn:tn+vl]
    idx_test = idx_random[tn+vl:tn+vl+tt]

    if need_orig:
        adj_orig = aug_normalized_adjacency(adj, need_orig=True)
        adj_orig = sparse_mx_to_torch_sparse_tensor(adj_orig).float()
        if cuda:
            adj_orig = adj_orig.cuda()

    adj, features = preprocess_citation(adj, features, normalization)

    # porting to pytorch
    features = torch.FloatTensor(np.array(features.todense() if sp.issparse(features) else features)).float()
    labels = torch.LongTensor(labels)
    labels = torch.max(labels, dim=1)[1]
    adj = sparse_mx_to_torch_sparse_tensor(adj).float()
    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    if cuda:
        features = features.cuda()
        adj = adj.cuda()
        labels = labels.cuda()
        idx_train = idx_train.cuda()
        idx_val = idx_val.cuda()
        idx_test = idx_test.cuda()

    return [adj, adj_orig] if need_orig else adj, features, labels, idx_train, idx_val, idx_test

class bimodel(torch.nn.Module):
    def __init__(self, num_in, num_hid, num_out, num_node, time_steps, kappa=0.99, phi=F.relu, b_direct=False):
        super(bimodel, self).__init__()
        self.i = num_in
        self.h = num_hid
        self.o = num_out
        self.n = num_node
        self.t = time_steps
        self.k = kappa
        self.direct = b_direct

        self.phi = [F.relu]*self.t
        self.X_0 = Parameter(torch.zeros(self.h, num_node), requires_grad=False)
        self.W = nn.ParameterList([Parameter(torch.FloatTensor(self.h, self.h)) for i in range(self.t)])
#         self.W = nn.ParameterList([Parameter(torch.FloatTensor(self.h, self.h))])
#         self.Omega = nn.ParameterList([Parameter(torch.FloatTensor(self.h, self.i)) for i in range(self.t)])
        self.V = Parameter(torch.FloatTensor(self.h, self.i))
#         self.linear = nn.Linear(self.h, self.o)
        self.classifier = nn.Sequential(
#                                     nn.Dropout(p=0.3)
                                    # nn.Linear(self.h, self.h),
#                                     nn.BatchNorm1d(self.h),
                                    # nn.Softplus(),
                                    nn.Linear(self.h, self.o),
                                    nn.LogSoftmax(dim=1)
                                    )
        self.init()

    def init(self):
#         stdv = 0.01
        for i in range(len(self.W)):
            stdv = 1. / (math.sqrt(self.W[i].size(1))* self.t)
            self.W[i].data.uniform_(0, stdv)
#             self.Omega[i].data.uniform_(-stdv, stdv)
#         stdv = 1. / self.W[0].size(1)
#         self.W[0].data.uniform_(-stdv, stdv)
        stdv = 1. /(math.sqrt(self.V.size(1))* self.t) 
        # stdv = 1
        self.V.data.uniform_(0, stdv)
        
    def project(self,X_list, A_list, A_rho):
        
        self.X_list = X_list
        self.A_list = A_list
        for i in range(len(self.W)):
            self.W[i] = projection_norm_inf(self.W[i], kappa=self.k / A_rho[i])
            
        
    def forward(self,Z):

        X_list = self.X_list
        A_list = self.A_list
        V = self.V
        W_list = self.W
        phi = F.relu

        for j in range(len(A_list)):
                W = W_list[j]
                A = A_list[j]
                B = torch.spmm(V, X_list[j])
                Z_ = W @ Z
                support = torch.spmm(A.T, Z_.T).T

                Z = phi(support + B)
        return Z
    
    def predict(self, X_list, A_list):
        # for i in range(len(self.W)):
        #     self.W[i] = projection_norm_inf(self.W[i], kappa=self.k / A_rho[i])
        
        V = self.V
        W_list = self.W
        phi = F.relu
        max_iter = 300
        tol = 1e-6
        
        device = W_list[0].device
        Z = torch.zeros(self.X_0.shape).to(device)
        
        status = 'max itrs reached'
        for i in range(max_iter):
            Z_old = Z
            for j in range(len(A_list)):
                    W = W_list[j]
                    A = A_list[j]
                    B = torch.spmm(V, X_list[j])
                    Z_ = W @ Z_old
                    support = torch.spmm(A.T, Z_.T).T

                    Z = phi(support + B)
            
            err = torch.norm(Z - Z_old, np.inf)
            if err < tol:
                status = 'converged'
                break
            
        if status == 'max itrs reached':
                
                print('Forward Not Converge! Error: %3.5f, tol: %3.5f' % (err, tol))
            
        Y_hat = self.classifier(Z.T)
        return Y_hat, Z.T

In [ ]:
def load_time_chain():
    r = np.random.RandomState(42)
    c = 2 # num of classes
    nc = 10 # num of each class
    n = 10 # nodes of chain
    l = 10 # length of chain
    f = 100 # feature dimension
    tn = 2  # train nodes
    vl = 1 # val nodes
    tt = 7 # test nodes
    noise = 0.00

    X_list = []
    A_list = []
    Y_list = []
    E_list = []
    for i in range(nc):
        features = r.uniform(-noise, noise, size=(l, n, f))
        features[0,0,0] += 1 
        X = [torch.tensor(features[j,:,:]) for j in range(l)]
        labels = torch.ones(n, dtype=torch.long)
        
        edge_index = torch.tensor(np.array([np.arange(10-1), np.arange(1, 10)]))
        E = [edge_index] * l
        chain_adj = sp.coo_matrix((np.ones(n-1), (np.arange(n-1), np.arange(1, n))), shape=(n, n))
        chain_adj = aug_normalized_adjacency(chain_adj)
        A = [chain_adj] * l

        X_list.append(X)
        A_list.append(A)
        Y_list.append(labels)
        E_list.append(E)

    for i in range(nc):
        features = r.uniform(-noise, noise, size=(l, n, f))
        X = [torch.tensor(features[j,:,:]) for j in range(l)]
        labels = torch.zeros(n, dtype=torch.long)

        edge_index = torch.tensor(np.array([np.arange(10-1), np.arange(1, 10)]))
        E = [edge_index] * l
        chain_adj = sp.coo_matrix((np.ones(n-1), (np.arange(n-1), np.arange(1, n))), shape=(n, n))
        chain_adj = aug_normalized_adjacency(chain_adj)
        A = [chain_adj] * l

        X_list.append(X)
        A_list.append(A)
        Y_list.append(labels)
        E_list.append(E)

    return X_list, A_list, Y_list, E_list
            
def load_time_chain2(l=10):
    r = np.random.RandomState(42)
    c = 2 # num of classes
    nc = 20 # num of each class
    n = 10 # nodes of chain
    # l = 5 # length of chain
    f = 4 # feature dimension
    tn = 20  # train nodes
    vl = 100 # val nodes
    tt = 200 # test nodes
    noise = 0.00

    X_list = []
    A_list = []
    Y_list = []
    E_list = []
    
    chain_adj = sp.coo_matrix((np.ones(n-1), (np.arange(n-1), np.arange(1, n))), shape=(n, n))
    adj = sp.block_diag([chain_adj for _ in range(c*nc)]) # square matrix N = c*n*l

    features = r.uniform(-noise, noise, size=(c, nc, n, f))
    #features = np.zeros_like(features)
    features[:, :, 0, :c] += np.eye(c).reshape(c, 1, c) # add class info to the first node of chains.
    features = features.reshape(-1, f)

    labels = np.eye(c).reshape(c, 1, 1, c).repeat(nc, axis=1).repeat(n, axis=2) # one-hot labels
    labels = labels.reshape(-1, c)
    labels = torch.LongTensor(labels)
    labels = torch.max(labels, dim=1)[1]
    
    adj, features = preprocess_citation(adj, features, "AugNormAdj")
    X_list.append(features)
    A_list.append(adj)
    Y_list.append(labels)
    
    idx_random = np.arange(c*nc*n)
    r.shuffle(idx_random)
    idx_train = idx_random[:tn]
    idx_val = idx_random[tn:tn+vl]
    idx_test = idx_random[tn+vl:tn+vl+tt]
    
    for i in range(l-1):
        chain_adj = sp.coo_matrix((np.ones(n-1), (np.arange(n-1), np.arange(1, n))), shape=(n, n))
        adj = sp.block_diag([chain_adj for _ in range(c*nc)]) # square matrix N = c*n*l

        features = r.uniform(-noise, noise, size=(c, nc, n, f))
        features = features.reshape(-1, f)

        adj, features = preprocess_citation(adj, features, "AugNormAdj")
        X_list.append(features)
        A_list.append(adj)
        

    return X_list, A_list, Y_list, idx_train, idx_val, idx_test

def load_time_chain3(l=10):
    r = np.random.RandomState(42)
    c = 2 # num of classes
    nc = 1 # num of each class
    n = 200 # nodes of chain
    # l = 5 # length of chain
    f = 10 # feature dimension
    tn = 200  # train nodes
    vl = 100 # val nodes
    tt = 100 # test nodes
    noise = 0.01

    X_list = []
    A_list = []
    Y_list = []
    E_list = []
    
    
    adj = sp.coo_matrix((np.ones(c*n-1), (np.arange(c*n-1), np.arange(1, c*n))), shape=(c*n, c*n))
    # adj = sp.block_diag([chain_adj for _ in range(c*nc)]) # square matrix N = c*n*l

    features = r.uniform(-0, 0, size=(c, nc, n, f))
    #features = np.zeros_like(features)
    features[:, 0, :, :c] += np.eye(c).reshape(c, 1, c) # add class info to the first node of chains.
    features = features.reshape(-1, f)
    

    labels = np.eye(c).reshape(c, 1, 1, c).repeat(nc, axis=1).repeat(n, axis=2) # one-hot labels
    labels = labels.reshape(-1, c)
    labels = torch.LongTensor(labels)
    labels = torch.max(labels, dim=1)[1]
    
    adj, features = preprocess_citation(adj, features, "AugNormAdj")
    X_list.append(features)
    A_list.append(adj)
    Y_list.append(labels)
    
    idx_random = np.arange(c*nc*n)
    r.shuffle(idx_random)
    idx_train = idx_random[:tn]
    idx_val = idx_random[tn:tn+vl]
    idx_test = idx_random[tn+vl:tn+vl+tt]
    
    for i in range(l-1):
        adj = sp.coo_matrix((np.ones(c*n-1), (np.arange(c*n-1), np.arange(1, c*n))), shape=(c*n, c*n))
        # adj = sp.block_diag([chain_adj for _ in range(c*nc)]) # square matrix N = c*n*l

        features = noise*r.uniform(-noise, noise, size=(c, nc, n, f))

        features = features.reshape(-1, f)

        adj, features = preprocess_citation(adj, features, "AugNormAdj")
        X_list.append(features)
        A_list.append(adj)
        

    return X_list, A_list, Y_list, idx_train, idx_val, idx_test

def clique_edge_index_ud(n):
    rows, cols = np.meshgrid(np.arange(n), np.arange(n))
    upper_tri_mask = np.triu(np.ones((n, n)), k=1).astype(bool)
    lower_tri_mask = np.tril(np.ones((n, n)), k=-1).astype(bool)
    rows = np.concatenate([rows[upper_tri_mask], rows[lower_tri_mask]], axis=0)
    cols = np.concatenate([cols[upper_tri_mask], cols[lower_tri_mask]], axis=0)
    return (rows, cols)

def load_toy_data(l=10):
    r = np.random.RandomState(42)
    c = 2 # num of classes
    nc = 1 # num of each class
    n = 5 # nodes of chain
    # l = 5 # length of chain
    f = c*n # feature dimension
    tn = 2  # train nodes
    vl = 1 # val nodes
    tt = 7 # test nodes
    noise = 1

    X_list = []
    A_list = []
    Y_list = []
    E_list = []
    
    rows, cols = clique_edge_index_ud(c*n)
    adj = sp.coo_matrix((np.ones(len(rows)), (rows, cols)), shape=(c*n, c*n))
    features = np.eye(c*n)
    
    labels = np.eye(c*n)
    labels = torch.LongTensor(labels)
    labels = torch.max(labels, dim=1)[1]
    
    adj, features = preprocess_citation(adj, features, "AugNormAdj")
    X_list.append(features)
    A_list.append(adj)
    Y_list.append(labels)
    
    idx_random = np.arange(c*nc*n)
    r.shuffle(idx_random)
    idx_train = idx_random[:tn]
    idx_val = idx_random[tn:tn+vl]
    idx_test = idx_random[tn+vl:tn+vl+tt]
    
    for i in range(l-1):
        features = noise*np.random.rand(c, nc, n, f)

        features = features.reshape(-1, f)

        adj, features = preprocess_citation(adj, features, "AugNormAdj")
        X_list.append(features)
        A_list.append(adj)
        

    return X_list, A_list, Y_list, idx_train, idx_val, idx_test

def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def clique_edge_index(n):
    rows, cols = np.meshgrid(np.arange(n), np.arange(n))
    upper_tri_mask = np.triu(np.ones((n, n)), k=1).astype(bool)
    return (rows[upper_tri_mask], cols[upper_tri_mask])

def clique_edge_index_ud(n):
    rows, cols = np.meshgrid(np.arange(n), np.arange(n))
    upper_tri_mask = np.triu(np.ones((n, n)), k=1).astype(bool)
    lower_tri_mask = np.tril(np.ones((n, n)), k=-1).astype(bool)
    rows = np.concatenate([rows[upper_tri_mask], rows[lower_tri_mask]], axis=0)
    cols = np.concatenate([cols[upper_tri_mask], cols[lower_tri_mask]], axis=0)
    return (rows, cols)
            

In [ ]:
# Load data
from scipy.sparse import coo_array
import time

device = 'cuda:7'
X_list, A_list, Y, idx_train, idx_val, idx_test = load_toy_data(10)

labels = Y[0].to(device)
X_list = [torch.from_numpy(X) for X in X_list]
X_list = [X.type(torch.FloatTensor).to(device).T for X in X_list]
num_nodes = X_list[0].shape[1]
A_list_orig = [sparse_mx_to_torch_sparse_tensor(A).to(device) for A in A_list]
A_list_normalized = [sparse_mx_to_torch_sparse_tensor(aug_normalized_adjacency(A)).to(device) for A in A_list]
A_rho = [get_spectral_rad(adj) for adj in A_list_normalized]

In [ ]:
############### IDGNN ##################

model = bimodel(10, 2, 10, 10, 10, kappa=0.95).to(device)
optimizer = torch.optim.Adam(model.parameters(), 
                       lr=0.01)
criterion = torch.nn.NLLLoss()

Z_0 = torch.zeros(2*10)
V_0 = torch.zeros(2*10)
eta_1 = 0.9
eta_2 = 0.01



tr = []
tt = []
val = []
best = 0
best_test = 0
patient = 0
now = time.time()

loss_1 = []
loss_2 = []
acc_1 = []
for i in range(2000):
        model.train()
        model.project(X_list, A_list_normalized, A_rho)
        Z_1 = Z_0.to(device).requires_grad_(True)
        Z = model(Z_1.view(2, -1))
        
        with torch.no_grad():
            re_loss = torch.sum((Z_1 - Z.flatten())**2)

        Y_hat = model.classifier(Z.T)

        upper_loss = criterion(Y_hat, labels)
        lower_loss = ((Z_1-Z.reshape(-1))**2).sum()
        
        
        z_grad = torch.autograd.grad(upper_loss, Z_1, retain_graph=True)[0]
        
        Z_0 = (1 - eta_1)*Z_0 + eta_1*Z.reshape(-1).detach().cpu()

        g_z_grad = torch.autograd.grad(lower_loss, Z_1, retain_graph=True, create_graph=True)[0]

        hv = torch.inner(g_z_grad,V_0.to(device))
        phi_v = torch.autograd.grad(hv, Z_1, retain_graph=True)[0]
        V_0 = V_0 - eta_2*phi_v.cpu() + eta_2*z_grad.cpu()

        loss = upper_loss - torch.inner(g_z_grad,V_0.to(device))

        loss.backward()
        if torch.isnan(model.W[0].grad).sum():
            import ipdb; ipdb.set_trace()

        optimizer.step()
        optimizer.zero_grad()
        
        acc_train = accuracy(Y_hat, labels).item()

        print(f'epoch: {i} loss: {loss.item():.4f} reconstruct loss: {re_loss.item():.4f} acc: {acc_train:.4f}')
        loss_1.append(upper_loss.item())
        loss_2.append(re_loss.item())
        acc_1.append(acc_train)


In [ ]:
Z = Z_0.view(2,10).T.cpu().detach().numpy()

import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.heatmap(Z, linewidth=0.2)
plt.show()

In [ ]:
################## TGCN & GRU-GCN ######################

try:
    from tqdm import tqdm
except ImportError:
    def tqdm(iterable):
        return iterable

import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import GConvGRU, TGCN
from sklearn.metrics import accuracy_score



class RecurrentGCN(torch.nn.Module):
    def __init__(self, num_hid, node_features, num_layers, num_class):
        super(RecurrentGCN, self).__init__()
        self.recurrent = nn.ModuleList([TGCN(node_features, num_hid) if i==0 else TGCN(node_features, num_hid) for i in range(num_layers) ])
        # self.recurrent = nn.ModuleList([GConvGRU(node_features, num_hid) if i==0 else GConvGRU(node_features, num_hid) for i in range(num_layers) ])
        
        self.linear = torch.nn.Linear(num_hid, num_class)
        self.log = nn.LogSoftmax(dim=1)
        self.layers = num_layers

    def forward(self, X_list, A_list, prev_hidden_state):
        x = X_list[0].T
        edge_index = A_list[0].coalesce().indices()
        edge_weight = A_list[0].coalesce().values()
        h = self.recurrent[0](x, edge_index, edge_weight, prev_hidden_state)
        for i in range(1, self.layers):
            x = X_list[i].T
            edge_index = A_list[i].coalesce().indices()
            edge_weight = A_list[i].coalesce().values()
            h = self.recurrent[i](x, edge_index, edge_weight, h)
        return h
    
    def predict(self, h_0):
        y = F.relu(h_0)
        y = self.linear(y)
        y = self.log(y)
        return y
    
device = 'cuda:7'
model = RecurrentGCN(node_features = 10, num_hid = 2, num_layers=len(A_list), num_class=10).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

loss = torch.nn.NLLLoss()

model.train()
losses = []
accs = []
y = Y[0]
for epoch in range(2000):
    h = None
    h = model(X_list, A_list_orig, h)
    y_hat = model.predict(h)
    cost = loss(y_hat, y.to(device))
    cost.backward()
    optimizer.step()
    optimizer.zero_grad()
    acc = accuracy_score(y, y_hat.argmax(1).cpu())
    accs.append(acc)
    print(f'cost: {cost:.4f}, acc {acc:.4f}')
    losses.append(cost.item())
model.eval()


h = None
h = model(X_list, A_list, h)
y_hat = model.predict(h)
acc = accuracy_score(y, y_hat.argmax(1).cpu())
print("Acc: {:.4f}".format(acc))

In [ ]:
h = h.cpu().detach().numpy()

import matplotlib.pyplot as plt
import seaborn as sns
ax = sns.heatmap(h, linewidth=0.2)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.title('Training loss')
plt.plot(losses, label='baseline')
plt.plot(loss_1, label='IDGNN')
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.title('Training loss')
plt.plot(accs, label='baseline')
plt.plot(acc_1, label='IDGNN')
plt.legend()
plt.show()